In [ ]:
%%capture
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

#Import

In [ ]:
from google.colab.drive import mount
import numpy as np
import os
from time import time
from MI_EEG_ClassMeth.FeatExtraction import Power_based_Connectivities, Phase_based_Connectivities, MotifSynchronization
from MI_EEG_ClassMeth.Preprocessing import Flatten
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score
from joblib import load as jload

#Mount Drive

In [ ]:
mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/evaluation/')

#Power based Connectivities

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/evaluation/'

seed = 23

subjects = np.arange(9) + 1

n_metrics = 3
n_hyparams = 4

pbc = Power_based_Connectivities()
flatt = Flatten()

# ------------------------------------------------------------------------------
t_i = time()
for type_con in ['pearson', 'spearman', 'gfc']:
  results = np.zeros((n_metrics, subjects.shape[0], n_hyparams))

  for sbj in subjects:
    y_dir = os.path.join(data_dir , 'labels/evaluation/')
    y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

    idx_hyparams=0
    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/' + 'evaluation/' + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = pbc.transform(X, type_con=type_con, gammad=1)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)
        clf = jload(models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
        y_pred = clf.predict(flat_C_xx_w_f)

        #Metrics
        results[:, sbj-1, idx_hyparams] = accuracy_score(y, y_pred), cohen_kappa_score(y, y_pred), roc_auc_score(y, clf.predict_proba(flat_C_xx_w_f), average='macro', multi_class='ovo')
        idx_hyparams += 1
  
  np.save(results_dir + type_con, results)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 6.44 m


#Phase based Connectivities

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/evaluation/'

seed = 23

subjects = np.arange(9) + 1

n_metrics = 3
n_hyparams = 4

pbc = Phase_based_Connectivities()
flatt = Flatten()

# ------------------------------------------------------------------------------
t_i = time()
for type_con in ['coh', 'plv', 'pli', 'wpli']:
  results = np.zeros((n_metrics, subjects.shape[0], n_hyparams))

  for sbj in subjects:
    y_dir = os.path.join(data_dir , 'labels/evaluation/')
    y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

    idx_hyparams=0
    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/' + 'evaluation/' + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = pbc.transform(X, type_con=type_con)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)
        clf = jload(models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
        y_pred = clf.predict(flat_C_xx_w_f)

        #Metrics
        results[:, sbj-1, idx_hyparams] = accuracy_score(y, y_pred), cohen_kappa_score(y, y_pred), roc_auc_score(y, clf.predict_proba(flat_C_xx_w_f), average='macro', multi_class='ovo')
        idx_hyparams += 1
  
  np.save(results_dir + type_con, results)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 29.83 m


#Information Based Connectivities

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/BCICIV2a_preprocessing/results/evaluation/'

seed = 23

subjects = np.arange(9) + 1

n_metrics = 3
n_hyparams = 4

mt = MotifSynchronization()
flatt = Flatten()

# ------------------------------------------------------------------------------
t_i = time()
for type_con in ['motif']:
  results = np.zeros((n_metrics, subjects.shape[0], n_hyparams))

  for sbj in subjects:
    y_dir = os.path.join(data_dir , 'labels/evaluation/')
    y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

    idx_hyparams=0
    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/' + 'evaluation/' + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = mt.transform(X)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)
        clf = jload(models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
        y_pred = clf.predict(flat_C_xx_w_f)

        #Metrics
        results[:, sbj-1, idx_hyparams] = accuracy_score(y, y_pred), cohen_kappa_score(y, y_pred), roc_auc_score(y, clf.predict_proba(flat_C_xx_w_f), average='macro', multi_class='ovo')
        idx_hyparams += 1
  
  np.save(results_dir + type_con, results)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 4.72 m
